In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt

In [18]:
xtrain_all = np.loadtxt('Data/train_data.csv', delimiter=',')
ytrain_all = np.loadtxt('Data/train_labels.csv', delimiter=',')
xtest = np.loadtxt('Data/test_data.csv', delimiter=',')
scaler = StandardScaler()
xtrain = scaler.fit_transform(xtrain_all)
xtest = scaler.fit_transform(xtest)
xtrain, xtrain2, ytrain, ytrain2 = train_test_split(xtrain_all, ytrain_all,
                                                    test_size=0.20, random_state=42)

m = xtrain.shape[0]
chc = np.random.choice(m, m/10)
xtrain_s = xtrain[chc,:]
ytrain_s = ytrain[chc]

/Users/janfan/.pyenv/versions/anaconda3-4.2.0/lib/python3.5/site-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [12]:
print(xtrain_s.shape)
print(ytrain_s.shape)

(349, 264)
(349,)


In [ ]:
# line search on k, i.e. number of top features (k/264)
for k in np.linspace(10, 220, 25):  
    k = int(k)
    model = LogisticRegression()
    rfe = RFE(model, k, step=2)
    fit = rfe.fit(xtrain_s, ytrain_s)
    xtrain_feat_k = xtrain[:,fit.support_]
    xtrain2_feat_k = xtrain2[:,fit.support_]
    model_feat_k = LogisticRegression()
    fit_feat_k = model_feat_k.fit(xtrain_feat_k, ytrain)
    print('number of top features: ' + str(k))
    print(fit_feat_k.score(xtrain2_feat_k, ytrain2))

In [17]:
k = 158
model = LogisticRegression()
rfe = RFE(model, k, step=2)
fit = rfe.fit(xtrain_s, ytrain_s)
model_feat_k = LogisticRegression()
fit_feat_k = model_feat_k.fit(xtrain[:,fit.support_], ytrain)
print('number of top features: ' + str(k))
print(fit_feat_k.score(xtrain2[:,fit.support_], ytrain2))

number of top features: 158
0.636884306987


In [15]:
k = 158
model = LogisticRegression()
rfe = RFE(model, k, step=2)
fit = rfe.fit(xtrain_s, ytrain_s)

model_feat_k = LogisticRegression()
fit_feat_k = model_feat_k.fit(xtrain_all[:,fit.support_], ytrain_all)

print('number of top features: ' + str(k))
print(fit_feat_k.score(xtrain_all[:,fit.support_], ytrain_all))
fit_feat_k.predict(xtest[:,fit.support_])

number of top features: 158
0.679119871648


array([  4.,   4.,   8., ...,  10.,   4.,   8.])

In [16]:
ypred = fit_feat_k.predict(xtest[:,fit.support_])
subm = pd.DataFrame(ypred, columns=['Sample_label'])
subm.index += 1
subm = subm.astype(int)
subm.to_csv('Solutions/logistic_featureSelection_2011.csv',
            index_label='Sample_id')